# **negativas**
> _versão 2_

O presente Notebook tem como objetivo a implementação de um código de busca e classificação de negações sentenciais em arquivos de entrevistas transcritas da amostra _Deslocamentos_, do Banco de Dados Falares Sergipanos¹.

O arquivo está disposto da seguinte forma: a **parte 1** trata da instalação dos componentes necessários para a execução do código; a **parte 2** traz a implementação das funções que serão utilizadas pela ferramenta; o código responsável pela análise (busca e classificação) encontra-se na **parte 3**; por fim, a **parte 4** trata do armazenamento dos dados em forma de planilhas.

</br>

---

1. FREITAG, R. M. Ko. Banco de dados falares sergipanos. Working Papers em
Linguística, v. 14, n. 2, p. 156-164, 2013. DOI: https://doi.org/10.5007/1984-8420.2013v14n2p156


> As principais funcionalidades foram desenvolvidas através da utilização do [spaCy](https://spacy.io/usage), biblioteca de Python para Processamento de Linguagem Natural.

## **1. Instalações e importações necessárias**
Nessa seção são instaladas e importadas todas as bibliotecas e componentes necessários para o desenvolvimento e funcionamento dos códigos.

In [ ]:
!pip install --quiet spacy
!python -m spacy download pt_core_news_lg

**Usando o Google Drive**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import string
import re

## **2. Implementando funções**
Aqui são implementadas duas funções que farão parte do código principal. Uma para a limpeza do texto e outra para a extração dos metadados de cada entrevista.

### **2.1. Função de limpeza**
A função de limpeza irá receber o texto do arquivo em processamento e limpá-lo, removendo stopwords e pontuações.

In [ ]:
def limpa_str(texto):
    nlp = spacy.blank('pt')
    stoplist = ['eh', 'hes', 'RISOS', 'risos', 'BARULHO', 'INTERVENIENTE', 'PIGARRO', 'pigarro', 'CHORO', 'RUÍDO', 'RUÍDOS','est']

    texto = texto.encode('utf-8','ignore').decode()
    texto = re.sub('[%s]' % re.escape(string.punctuation), ' ', texto)
    doc = nlp(texto)
    palavras = [palavra for palavra in texto.split() if not re.search([p for p in stoplist], palavra)]
    for token in doc:
        if token.text not in stoplist:
            palavras.append(token.text)
    texto_limpo = (' ').join(palavras)

    return texto_limpo

### **2.2. Função de extração do cabeçalho**
Cada arquivo de entrevista transcrita possui um cabeçalho com dados sobre as pessoas entrevistadas e entrevistadoras. A função a seguir realiza a extração desses dados.

In [ ]:
def extrai_cabecalho(entrevista, dado):
    cabecalho_arquivo = []
    conteudo = entrevista.split('\n')

    for linha in range (0, 12):
        texto = conteudo[linha].strip('\n').upper().split()
        for palavra in range(len(texto)):
            documentador = ['DOCUMENTADOR(A):', 'DOCUMENTADOR:']
            if texto[palavra] == dado:
                return texto[palavra+1]
            elif texto[palavra] in documentador:
                return texto[palavra+1]

## **3. Código de busca e classificação**
O código seguinte é o principal da ferramenta, é nele onde estão implementados os comandos de busca e classificação das negações sentenciais.

Através dele, todas as entrevistas contidas no diretório serão processadas buscando pelas ocorrências que atendam as regras estabelecidas no `Matcher` da biblioteca spaCy e classificando-as com base nas etiquetas atribuídas à cada padrão, podendo ser:
* **pré-verbal**
* **pós-verbal**
* **dupla negação**

Finalizada a busca e classificação, os dados são armazenados em uma lista que dará origem ao DataFrame, contendo as informações do cabeçalho da entrevista, a ocorrência, a classificação da negação e o seu contexto.

Além da estrutura contendo os dados analisados, será criado outro DataFrame para armazenar as estatísticas de cada entrevista, como o nº total de negações, quantidade de negações para cada etiqueta e suas porcentagens em relação ao total.

In [ ]:
#DataFrame de análise e classificação das negações
colunas_neg = ["Documentador(a)", "Inf", "Sexo", "Ocorrência", "Classificação", "Contexto"]
linhas_neg = []

#DataFrame contendo as estatísticas
colunas_est = ["Arquivo", "Inf", "Nº de 'nãos'", "Negações", "Pré-verbal", "% pré-verbal", "Pós-verbal", "% pós-verbal", "Dupla negação", "% dupla negação"]
linhas_est = []

#Lista todos os arquivos contidos na pasta data
nomes_arq = !ls 'data'

nlp = spacy.blank("pt")
nlp = spacy.load("pt_core_news_lg")

for nome in nomes_arq:
    nome_refatorado = nome.replace("'", "")
    arquivo =f'data/{nome_refatorado}'

    entrevista = open(arquivo, encoding='utf-8').read()

    documentador = extrai_cabecalho(entrevista, "DOCUMENTADORA:")
    inf = extrai_cabecalho(entrevista, "INF:")
    sexo = extrai_cabecalho(entrevista, "SEXO:")

    conteudo_arquivo = limpa_str(open(arquivo).read())
    doc = nlp(limpa_str(entrevista))

    matcher = Matcher(vocab=nlp.vocab)

    nao_pre_verbal = [{"TEXT": "não"}, {"POS": {"IN": ["VERB", "AUX"]}}, {"TEXT": "não", "OP": "!"}, {"POS": {"NOT_IN": ["VERB", "AUX"]}}]
    nao_pos_verbal = [{"TEXT": "não", "OP": "!"}, {}, {"POS": {"IN": ["VERB", "AUX"]}}, {"TEXT": "não"}]
    dupla_negacao = [{"TEXT": "não"}, {"POS": {"IN": ["VERB", "AUX"]}, "OP": "+"}, {"TEXT": "não"}, {"POS": {"IN": ["VERB", "AUX"]}, "OP": "!"}]

    matcher.add('dupla negação', [dupla_negacao])
    matcher.add('pré-verbal', [nao_pre_verbal])
    matcher.add('pós-verbal', [nao_pos_verbal])

    matches = matcher(doc)

    nao = len(re.findall(r'\s?não\s?', conteudo_arquivo))
    dupla_neg = 0
    pre_verbal = 0
    pos_verbal = 0
    negacoes = 0

    for match_id, start, end in matches:
        ocorrencia = doc[start:end]
        classificacao = nlp.vocab.strings[match_id]
        contexto = doc[start-5:end+4]
        linhas_neg.append([documentador, inf, sexo, ocorrencia, classificacao, contexto])

        if classificacao == 'dupla negação':
            dupla_neg += 1
            negacoes += 1
        elif classificacao == 'pré-verbal':
            pre_verbal += 1
            negacoes += 1
        elif classificacao == 'pós-verbal':
            pos_verbal += 1
            negacoes += 1

    #Armazenando estatísticas básicas
    linhas_est.append([nome_refatorado, inf, nao, negacoes, pre_verbal, ((pre_verbal / negacoes) * 100), pos_verbal, ((pos_verbal / negacoes) * 100), dupla_neg, ((dupla_neg / negacoes) * 100)])


## **4. Armazenando os resultados**


### **4.1. Construindo e verificando os DataFrames**
Aqui, os DataFrames são criados e algumas das suas informações são exibidas para verificar se tudo funcionou corretamente.

**Ocorrências e classificações**

In [ ]:
df_neg = pd.DataFrame(linhas_neg, columns = colunas_neg)

df_neg.info()
df_neg.head()

**Estatísticas básicas**

In [ ]:
df_est = pd.DataFrame(linhas_est, columns = colunas_est)

df_est.info()
df_est.head()

### **4.2. Salvando os DataFrames**
O salvamento dos dados pode ocorrer tanto em arquivo `.csv` quanto arquivo excel, `.xlsx`, e o nome do seu arquivo pode ser alterado na váriavel `NOME_ARQUIVO`.

> **Observação:** ao salvar os dados, lembre de movê-los para uma pasta de sua preferência em seu Drive ou baixá-lo na sua máquina, pois quando o Google Colab encerra a conexão, os arquivos salvos no ambiente são perdidos.

**Ocorrências e classificações**

In [ ]:
from datetime import date
NOME_ARQUIVO = 'negativas_amostra2020'

#Salva em .csv
df_neg.to_csv(f"teste_{date.today().strftime('%d-%m-%Y')}_{NOME_ARQUIVO}.csv", encoding='UTF-8')

#Salva em excel
#df_neg.to_excel(f"{NOME_ARQUIVO}.xlsx", encoding='UTF-8')

**Estatísticas**

In [ ]:
#Salva em .csv
df_est.to_csv(f"Estatísticas - {NOME_ARQUIVO}.csv", encoding='UTF-8')

#Salva em excel
#df_est.to_excel(f"Estatísticas - {NOME_ARQUIVO}.xlsx", encoding='UTF-8')